In [2]:
from bs4 import BeautifulSoup
import urllib.request
import urllib.parse
import urllib
import re
import os
try:
    from urllib.request import Request, urlopen  # Python 3
except:
    from urllib2 import Request, urlopen  # Python 2

def get_urls_from_divs(divs):
    return list(map(lambda div: re.search(r"url\(//([\w\d\./]+)", str(div)).group().replace("url(//", "http://"), divs))

def get_url_imgs_flickr(queries):
    queries = [q.replace(" ", "%20") for q in queries]
    base_url = "https://www.flickr.com/search/?orientation=square&view_all=1&text="
    urls = [base_url + q for q in queries]
    results = []
    
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'

    headers = {'User-Agent': user_agent}
    for url in urls:
        req = Request(url, headers=headers)
        webpage = urlopen(req).read()

        soup = BeautifulSoup(webpage)
        mydivs = soup.findAll("div", { "class" : "photo-list-photo-view" })
        results += get_urls_from_divs(mydivs)
    return results


def get_urls_imgs_google(queries):
    queries = [q.replace(" ", "+") for q in queries]
    base_url = "https://www.google.com/search?source=lnms&tbm=isch&sa=X&q="
    urls = [base_url + q for q in queries]
    results = []
    
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'

    headers = {'User-Agent': user_agent}
    for url in urls:
        req = Request(url, headers=headers)
        webpage = urlopen(req).read()

        soup = BeautifulSoup(webpage)
        mydivs = [x.find("img")["src"] for x in soup.findAll("a") if x.find("img") != None]
        results += mydivs
    return results  

def download_imgs(path, class_name, url_imgs):
    cmd = "mkdir "+path+class_name
    os.system(cmd)
    for url in url_imgs:
        if "images?q=tbn:" in url:
            urllib.request.urlretrieve(url,path+class_name+"/"+url.split("images?q=tbn:")[-1] + ".jpg" )
        else:
            urllib.request.urlretrieve(url,path+class_name+"/"+url.split("/")[-1] )
        
def pipeline(queries_dict, path):
    for class_name, queries in list(queries_dict.items()):
        urls = set(get_url_imgs_flickr(queries) + get_urls_imgs_google(queries))
        download_imgs(path, class_name, urls)
        print("saved " + class_name)

In [5]:
q_d = {"Cocacola": ["cocacola", "coca cola", 
                    "cocacola sign", "coca cola sign", 
                    "Merchandising cocacola", 
                    "coca cola brand", "cocacola drink"],
       "Pepsi":["pepsi", "pepsi sign",
                "pepsi drink","Merchandising pepsi",
                "pepsi brand"],
       "Nike":["nike","nike shoes", "nike brand", 
               "Merchandising nike", "nike footwear", 
               "nike clothing", "nike sport"],
       "LOreal": ["LOreal brand","LOreal commercial", 
                   "LOreal product", "LOreal merchandising"],
       "LouisVuitton":["Louis Vuitton bag", "Louis Vuitton brand",
                       "Louis Vuitton merchandising","Louis Vuitton",
                       "Louis Vuitton women", "louis vuitton ad"
                      ],
       "Samsung":["Samsung", "Samsung phone", "Samsung brand", 
                  "Samsung merchandising","Samsung ad",
                  "Samsung commercial","Samsung product" ]
      }

In [6]:
pipeline(q_d, "/Users/francescoferrari/Downloads/logo_dataset_6/")

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


saved Samsung
saved Cocacola
saved Pepsi
saved Nike
saved LOreal
saved LouisVuitton
